In [1]:
import requests
import json
import pyTigerGraph as tg
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import csv
import os.path

In [2]:
#conn = tg.TigerGraphConnection(host="localhost", graphname="MyGraph", username="tigergraph", password="tigergraph")
conn = tg.TigerGraphConnection(host="http://127.0.0.1",graphname="graph_drug_disease", username="tigergraph", password="tigergraph", apiToken="2aa016d747ede9gg6da3drslm98srfoj")


In [24]:
#conn.gsql("use graph mygraph")
#conn.gsql("CREATE VERTEX movie (PRIMARY_ID id UINT, name STRING, year UINT) WITH primary_id_as_attribute=\"true\"")


'Semantic Check Fails: The vertex name movie is used by another object! Please use a different name.\nFailed to create vertex types: [movie].'

In [25]:
#conn.gsql("CREATE VERTEX User (PRIMARY_ID user_id UINT, name STRING, age UINT, gender STRING, postalCode STRING)")

'Successfully created vertex types: [User].'

In [26]:
#conn.gsql("CREATE UNDIRECTED EDGE user_watch (FROM User, TO movie)")

'Successfully created edge types: [user_watch].'

In [34]:
#conn.gsql("USE GLOBAL CREATE UNDIRECTED EDGE user_x (FROM User, TO X)")

'Successfully created edge types: [user_x].'

In [35]:
#conn.gsql("CREATE GRAPH mygraph (*)")

'Semantic Check Fails: The graph name conflicts with another type or existing graph names! Please use a different name.\nThe graph mygraph could not be created!'

In [22]:
conn.gsql("use global drop all")

'Dropping all, about 1 minute ...\nAbort all active loading jobs\nTry to abort all loading jobs on graph graph_drug_disease, it may take a while ...\n[ABORT_SUCCESS] No active Loading Job to abort.\nResetting GPE...\nSuccessfully reset GPE and GSE\nStopping GPE GSE\nSuccessfully stopped GPE GSE in 0.006 seconds\nClearing graph store...\nSuccessfully cleared graph store\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.478 seconds\nEverything is dropped.'

In [3]:
# create a set of drugs
conn.gsql("CREATE VERTEX DRUG (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")
# create a set of diseases
conn.gsql("CREATE VERTEX DISEASES (PRIMARY_ID DiseaseName STRING) With primary_id_as_attribute=\"true\"")
# create a set of prescriptor
#conn.gsql("CREATE VERTEX PRESCRIBER (PRIMARY_ID Prscrbr_NPI UINT, Prscrbr_Last_Org_Name STRING, Prscrbr_First_Name STRING, Prscrbr_MI STRING, Prscrbr_Crdntls STRING, Prscrbr_Gndr STRING, Prscrbr_City STRING, Prscrbr_State_Abrvtn STRING, Prscrbr_State_FIPS UINT, Prscrbr_Zip5 UINT, Prscrbr_RUCA STRING, Prscrbr_RUCA_Desc STRING, Prscrbr_Cntry STRING, Prscrbr_Type STRING) With primary_id_as_attribute=\"true\"")
conn.gsql("CREATE VERTEX PRESCRIBER (PRIMARY_ID Prscrbr_NPI UINT,Prscrbr_Last_Org_Name STRING,Prscrbr_First_Name STRING,Prscrbr_MI STRING,Prscrbr_Crdntls STRING,Prscrbr_Gndr STRING,Prscrbr_City STRING ,Prscrbr_State_Abrvtn STRING,Prscrbr_State_FIPS STRING,Prscrbr_zip5 STRING) With primary_id_as_attribute=\"true\"")
 
# create a set of gene
conn.gsql("CREATE VERTEX GENE (PRIMARY_ID GeneID STRING, GeneSymbol STRING, GeneName STRING ,AltGeneIDs STRING,Synonyms STRING,BioGRIDIDs STRING,PharmGKBIDs STRING,UniProtIDs STRING) With primary_id_as_attribute=\"true\"")

# create a set of BEHAVIORAL
conn.gsql("CREATE VERTEX BEHAVIORAL (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")


# create a set of BIOLOGICAL
conn.gsql("CREATE VERTEX BIOLOGICAL (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of BEHAVIORAL
conn.gsql("CREATE VERTEX BEHAVIORAL (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of COMBINATION_PRODUCT
conn.gsql("CREATE VERTEX COMBINATION_PRODUCT (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of DEVICE
conn.gsql("CREATE VERTEX DEVICE (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of DIAGNOSTIC_TEST
conn.gsql("CREATE VERTEX DIAGNOSTIC_TEST (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of DIETARY_SUPPLEMENT
conn.gsql("CREATE VERTEX DIETARY_SUPPLEMENT (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")


# create a set of GENETIC
conn.gsql("CREATE VERTEX GENETIC (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of DIAGNOSTIC_TEST
conn.gsql("CREATE VERTEX DIAGNOSTIC_TEST (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of OTHER
conn.gsql("CREATE VERTEX OTHER (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of PROCEDURE
conn.gsql("CREATE VERTEX PROCEDURE (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"")

# create a set of RADIATION
conn.gsql("CREATE VERTEX RADIATION (PRIMARY_ID name STRING) With primary_id_as_attribute=\"true\"") 



# Edges 

# create a relation between PRESCRIBER and DRUG
conn.gsql("CREATE UNDIRECTED EDGE prescript_drug (FROM PRESCRIBER, TO DRUG)")
# create a relation between DRUG and DISEASES prescripted by a prescriptor
conn.gsql("CREATE UNDIRECTED EDGE is_prescripted_for (FROM DRUG, TO DISEASES)")
# create a relation between DRUG and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE drug_is_tested_for (FROM DRUG, TO DISEASES, filename STRING)")

# create a relation between GENE and DISEASES
conn.gsql("CREATE UNDIRECTED EDGE gene_disease_association (FROM GENE, TO DISEASES)")
# create a relation between DRUG and GENE
conn.gsql("CREATE UNDIRECTED EDGE drug_gene_association (FROM DRUG, TO GENE)")


# create a relation between BEHAVIORAL and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE behavioral_is_tested_for (FROM BEHAVIORAL, TO DISEASES, filename STRING)")


# create a relation between BIOLOGICAL and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE bio_is_tested_for (FROM BIOLOGICAL, TO DISEASES, filename STRING)")


# create a relation between COMBINATION_PRODUCT and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE combination_pro_is_tested_for (FROM COMBINATION_PRODUCT, TO DISEASES, filename STRING)")


# create a relation between DEVICE and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE device_is_tested_for (FROM DEVICE, TO DISEASES, filename STRING)")


# create a relation between DIAGNOSTIC_TEST and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE diagnostic_test_is_tested_for (FROM DIAGNOSTIC_TEST, TO DISEASES, filename STRING)")

# create a relation between DIETARY_SUPPLEMENT and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE dietary_is_tested_for (FROM DIETARY_SUPPLEMENT, TO DISEASES, filename STRING)")

# create a relation between GENETIC and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE genetic_is_tested_for (FROM GENETIC, TO DISEASES, filename STRING)")

# create a relation between OTHER and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE other_is_tested_for (FROM OTHER, TO DISEASES, filename STRING)")

# create a relation between PROCEDURE and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE procedure_is_tested_for (FROM PROCEDURE, TO DISEASES, filename STRING)")

# create a relation between RADIATION and DISEASES tested in the clinical trial
conn.gsql("CREATE UNDIRECTED EDGE radiation_is_tested_for (FROM RADIATION, TO DISEASES, filename STRING)")

print("Successfully created")

Successfully created


In [4]:
conn.gsql("CREATE GRAPH graph_drug_disease (*)")

'Stopping GPE GSE RESTPP\nSuccessfully stopped GPE GSE RESTPP in 31.143 seconds\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.066 seconds\nThe graph graph_drug_disease is created.'

In [5]:
conn.gsql('''
create loading job load_data for graph graph_drug_disease {

    define filename file_prescriber_2013="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY13_NPI.csv";
    define filename file_prescriber_2014="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY14_NPI.csv";
    define filename file_prescriber_2015="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY15_NPI.csv";
    define filename file_prescriber_2016="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY16_NPI.csv";
    define filename file_prescriber_2017="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_PTD_R19_P16_V10_D17_Prov.csv";
    define filename file_prescriber_2018="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY18_NPI.csv";
    define filename file_prescriber_2019="/home/sofiane/tigergraph_drug/data_for_graph/provider/MUP_DPR_RY21_P04_V10_DY19_NPI.csv";
    
    define filename file_gene="/home/sofiane/tigergraph_drug/data_for_graph/genes/data";
    

    define filename file1="/home/sofiane/tigergraph_drug/data_for_graph/diseases/diseases.csv";
    define filename file2="/home/sofiane/tigergraph_drug/data_for_graph/drugs/drugs.csv";
    define filename file3="/home/sofiane/tigergraph_drug/data_for_graph/drugs/BEHAVIORAL.csv";
    define filename file4="/home/sofiane/tigergraph_drug/data_for_graph/drugs/BIOLOGICAL.csv";
    define filename file5="/home/sofiane/tigergraph_drug/data_for_graph/drugs/COMBINATION_PRODUCT.csv";
    define filename file6="/home/sofiane/tigergraph_drug/data_for_graph/drugs/DEVICE.csv";
    define filename file7="/home/sofiane/tigergraph_drug/data_for_graph/drugs/DIAGNOSTIC_TEST.csv";
    define filename file8="/home/sofiane/tigergraph_drug/data_for_graph/drugs/DIETARY_SUPPLEMENT.csv";
    define filename file9="/home/sofiane/tigergraph_drug/data_for_graph/drugs/GENETIC.csv";
    define filename file10="/home/sofiane/tigergraph_drug/data_for_graph/drugs/OTHER.csv";
    
    define filename file11="/home/sofiane/tigergraph_drug/data_for_graph/drugs/PROCEDURE.csv";
    define filename file12="/home/sofiane/tigergraph_drug/data_for_graph/drugs/RADIATION.csv";
    
    define filename file13="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/BEHAVIORAL_disease_clinical_trials.csv";
    define filename file14="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/BIOLOGICAL_disease_clinical_trials.csv";
    define filename file15="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/COMBINATION_PRODUCT_disease_clinical_trials.csv";
    define filename file16="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/DEVICE_disease_clinical_trials.csv";
    define filename file17="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/DIAGNOSTIC_TEST_disease_clinical_trials.csv";
    define filename file18="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/DIETARY_SUPPLEMENT_disease_clinical_trials.csv";
    define filename file19="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/drug_disease_clinical_trials.csv";
    define filename file20="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/GENETIC_disease_clinical_trials.csv";
    define filename file21="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/OTHER_disease_clinical_trials.csv";
    define filename file22="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/PROCEDURE_disease_clinical_trials.csv";
    define filename file23="/home/sofiane/tigergraph_drug/data_for_graph/clinical_trials-relations/RADIATION_disease_clinical_trials.csv";
    
    
    define filename file_prescrit_drug_2013="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY13_NPIBN_4_prescrit_drug.csv";
    define filename file_prescrit_drug_2014="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY14_NPIBN_1_prescrit_drug.csv";
    define filename file_prescrit_drug_2015="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY15_NPIBN_1_prescrit_drug.csv";
    define filename file_prescrit_drug_2016="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY16_NPIBN_0_prescrit_drug.csv";
    define filename file_prescrit_drug_2017="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY17_NPIBN_1_prescrit_drug.csv";
    define filename file_prescrit_drug_2018="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY18_NPIBN_0_prescrit_drug.csv";
    define filename file_prescrit_drug_2019="/home/sofiane/tigergraph_drug/data_for_graph/prescrit_drug/MUP_DPR_RY21_P04_V10_DY19_NPIBN_1_prescrit_drug.csv";
    
    
    define filename file_drug_disease="/home/sofiane/tigergraph_drug/data_for_graph/drug-disease_associations/drug-disease.csv";
    define filename file_gen_disease="/home/sofiane/tigergraph_drug/data_for_graph/gen-disease-asso/gen-disease.csv";
    define filename file_drug_gen="/home/sofiane/tigergraph_drug/data_for_graph/chemical-gene-asso/drug_gen.csv";
    
    
    
    
    
    load file_prescriber_2013 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2014 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2015 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2016 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2017 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2018 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    load file_prescriber_2019 to VERTEX PRESCRIBER values ($0,$1,$2,$3,$4,$5,$9,$10,$11,$12) using header="true", separator=",";
    
    
    load file_gene to VERTEX GENE values ($2, $0, $1, $3, $4, $5, $6, $7) using header="false", separator=",";
    
    load file1 to VERTEX DISEASES values ($0) using header="false", separator=",";
    load file2 to VERTEX DRUG values ($0) using header="false", separator=",";
    load file3 to VERTEX BEHAVIORAL values ($0) using header="false", separator=",";
    load file4 to VERTEX BIOLOGICAL values ($0) using header="false", separator=",";
    load file5 to VERTEX COMBINATION_PRODUCT values ($0) using header="false", separator=",";
    load file6 to VERTEX DEVICE values ($0) using header="false", separator=",";
    load file7 to VERTEX DIAGNOSTIC_TEST values ($0) using header="false", separator=",";
    load file8 to VERTEX DIETARY_SUPPLEMENT values ($0) using header="false", separator=",";
    load file9 to VERTEX GENETIC values ($0) using header="false", separator=",";
    load file10 to VERTEX OTHER values ($0) using header="false", separator=",";
    load file11 to VERTEX PROCEDURE values ($0) using header="false", separator=",";
    load file12 to VERTEX RADIATION values ($0) using header="false", separator=",";
    
    
    
    load file13 to EDGE behavioral_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file14 to EDGE bio_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file15 to EDGE combination_pro_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file16 to EDGE device_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file17 to EDGE diagnostic_test_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file18 to EDGE dietary_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file19 to EDGE drug_is_tested_for values ($0, $1, $2) using header="false", separator=";";
    load file20 to EDGE genetic_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file21 to EDGE other_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file22 to EDGE procedure_is_tested_for values ($0, $1, $2) using header="false", separator=",";
    load file23 to EDGE radiation_is_tested_for values ($0, $1, $2) using header="false", separator=",";

    
    load file_prescrit_drug_2013 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2014 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2015 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2016 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2017 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2018 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    load file_prescrit_drug_2019 to EDGE prescript_drug values ($0, $1) using header="false", separator=";";
    
    
    load file_drug_disease to EDGE is_prescripted_for values ($0, $1) using header="false", separator=";";
    load file_gen_disease to EDGE gene_disease_association values ($0, $1) using header="false", separator=";";
    load file_drug_gen to EDGE drug_gene_association values ($0, $1) using header="false", separator=";";
    
    
    
}
run loading job load_data
''')



'Successfully created loading jobs: [load_data].\n[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]\n[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]\nStarting the following job, i.e.\nJobName: load_data, jobid: graph_drug_disease.load_data.file.m1.1647851470313\nLoading log: \'/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/graph_drug_disease/graph_drug_disease.load_data.file.m1.1647851470313.log\'\n\nJob "graph_drug_disease.load_data.file.m1.1647851470313" loading status\n[WAITING] m1 ( Finished: 0 / Total: 0 )\nJob "graph_drug_disease.load_data.file.m1.1647851470313" loading status\n[RUNNING] m1 ( Finished: 3 / Total: 41 )\n[LOADED]\n+---------------------------------------------------------------------------------------------------------------------------------------------------------+\n|                                                                                

In [78]:
#conn.gsql("tg_jaccard_batch ()")
#conn.gsql("CREATE QUERY query2(/* Parameters here */) FOR GRAPH graph_drug_disease { x  = SELECT v FROM PRESCRIBER:v  limit 5;  print x;}")

'Successfully created queries: [query2].'

In [6]:
#conn.getInstalledQueries(fmt="py")

In [115]:
#conn.gsql('tg box algos install tg_jaccard_nbor_ss')


#conn.gsql('RUN QUERY tg_pageRank("Page","Links_to",_,30,_,50,_,_,_)')

'Semantic Check Fails: The query tg_jaccard_nbor_ss cannot be found!\nQuery installation failed!'

In [322]:
#with open('tg_jaccard_nbor_ap_batch.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)

#with open('tg_jaccard_nbor_ss.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)


#with open('tg_common_neighbors.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)

#with open('getDISEASES.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)


with open('getDRUGS.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)

#with open('getPrescriber.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)

#with open('tg_random_walk_batch.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)


#with open('tg_node2vec.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)


# error with open('tg_fastRP.gsql', 'r') as file:
#gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)

# tg_cosine_nbor_ap_batch ne trouve pas de score car il manque valeur dans edge atrribute
#with open('tg_cosine_nbor_ap_batch.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)

# tg_embedding_cosine_sim ne donne pas des vrai similarité donne sim=0
#with open('tg_embedding_cosine_sim.gsql', 'r') as file:
#    gsql_file = file.read().rstrip()
#conn.gsql(gsql_file)




'Successfully created queries: [getDRUGS].'

In [6]:
with open('getDRUGS.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY getDRUGS')

'Start installing queries, about 1 minute ...\ngetDRUGS query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/getDRUGS\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [7]:
with open('getDISEASES.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY getDISEASES')

'Start installing queries, about 1 minute ...\ngetDISEASES query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/getDISEASES\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [8]:
with open('tg_common_neighbors.gsql', 'r') as file:
    gsql_file = file.read().rstrip()
conn.gsql(gsql_file)
conn.gsql('INSTALL QUERY tg_common_neighbors')

'Start installing queries, about 1 minute ...\ntg_common_neighbors query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/tg_common_neighbors?a=VALUE&a.type=VERTEX_TYPE&b=VALUE&b.type=VERTEX_TYPE&e_type=VALUE&[print_res=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [25]:
#conn.gsql('DROP QUERY tg_random_walk_batch')

"Successfully dropped queries on the graph 'graph_drug_disease': [tg_random_walk_batch]."

In [323]:
#conn.gsql('INSTALL QUERY tg_jaccard_nbor_ap_batch')
#conn.gsql('INSTALL QUERY getDISEASES')
conn.gsql('INSTALL QUERY getDRUGS')

#conn.gsql('INSTALL QUERY tg_common_neighbors')
#conn.gsql('INSTALL QUERY getPRESCRIBER')
#conn.gsql('INSTALL QUERY tg_random_walk_batch')
#conn.gsql('INSTALL QUERY tg_node2vec')
#conn.gsql('INSTALL QUERY tg_cosine_nbor_ap_batch')
#conn.gsql('INSTALL QUERY tg_embedding_cosine_similarity')





'Start installing queries, about 1 minute ...\ngetDRUGS query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/getDRUGS\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [3]:
print(conn.gsql('RUN QUERY getPRESCRIBER ()'))
getPRESCRIBER = []
with open("/home/tigergraph/getPRESCRIBER.txt") as file:
    for line in file: 
        line = line.strip() #or some other preprocessing
        getPRESCRIBER.append(line) #storing everything in memory!

len(getPRESCRIBER)

{
"error": false,
"message": "",
"version": {
"schema": 0,
"edition": "enterprise",
"api": "v2"
},
"results": []
}


1577043

In [9]:


conn.gsql('''
CREATE QUERY pattern_clinical_trials_drugs(String drug) FOR GRAPH graph_drug_disease API("v2") SYNTAX v2 {

  SetAccum<string> @@VertexSet_1_idList;
  SetAccum<vertex> @@VertexSet_1_vertexList;
  SetAccum<edge> @@FinalEdgeSet_3;
  SetAccum<edge> @@FinalEdgeSet_4;
  SetAccum<vertex<DRUG>> @@FinalVertexSet_1;
  SetAccum<vertex<DISEASES>> @@FinalVertexSet_2;

  @@VertexSet_1_idList += drug;
  @@VertexSet_1_vertexList = to_vertex_set(@@VertexSet_1_idList, "DRUG");
  VertexSet_1 (DRUG) = { @@VertexSet_1_vertexList };
  VertexSet_1 = 
        SELECT alias_schema_1
        FROM VertexSet_1:alias_schema_1 -(drug_is_tested_for:alias_schema_3)- DISEASES:alias_schema_2,
             VertexSet_1:alias_schema_1 -(is_prescripted_for:alias_schema_4)- DISEASES:alias_schema_2
        ACCUM @@FinalEdgeSet_3 += alias_schema_3,
              @@FinalEdgeSet_4 += alias_schema_4
        POST-ACCUM @@FinalVertexSet_1 += alias_schema_1
        POST-ACCUM @@FinalVertexSet_2 += alias_schema_2
        ;

  PRINT @@FinalEdgeSet_3;

  PRINT @@FinalEdgeSet_4;

  VertexSet_1 = { @@FinalVertexSet_1 };
  PRINT VertexSet_1["" as no_attributes];

  VertexSet_2 = { @@FinalVertexSet_2 };
  PRINT VertexSet_2["" as no_attributes];

}

''')



"Warning in query pattern_clinical_trials_drugs (WARN-8): line 150, col 8\nIn PRINT clause, it contains an expression that doesn't have\nattribute/vertex-attached accumulator of vertex set. It will be printed\nrepeatedly the number of times as the given vertex set size.\nWarning in query pattern_clinical_trials_drugs (WARN-8): line 153, col 8\nIn PRINT clause, it contains an expression that doesn't have\nattribute/vertex-attached accumulator of vertex set. It will be printed\nrepeatedly the number of times as the given vertex set size.\nSuccessfully created queries: [pattern_clinical_trials_drugs]."

In [10]:

conn.gsql('INSTALL QUERY pattern_clinical_trials_drugs')

'Start installing queries, about 1 minute ...\npattern_clinical_trials_drugs query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/pattern_clinical_trials_drugs?drug=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [11]:
conn.gsql('''
CREATE QUERY pattern_clinical_trials_disease(String disease) FOR GRAPH graph_drug_disease API("v2") SYNTAX v2 {

  SetAccum<string> @@VertexSet_2_idList;
  SetAccum<vertex> @@VertexSet_2_vertexList;
  SetAccum<edge> @@FinalEdgeSet_3;
  SetAccum<edge> @@FinalEdgeSet_4;
  SetAccum<vertex<DRUG>> @@FinalVertexSet_1;
  SetAccum<vertex<DISEASES>> @@FinalVertexSet_2;

  @@VertexSet_2_idList += disease;
  @@VertexSet_2_vertexList = to_vertex_set(@@VertexSet_2_idList, "DISEASES");
  VertexSet_2 (DISEASES) = { @@VertexSet_2_vertexList };
  VertexSet_1 = 
        SELECT alias_schema_1
        FROM DRUG:alias_schema_1 -(drug_is_tested_for:alias_schema_3)- VertexSet_2:alias_schema_2,
             DRUG:alias_schema_1 -(is_prescripted_for:alias_schema_4)- VertexSet_2:alias_schema_2
        ACCUM @@FinalEdgeSet_3 += alias_schema_3,
              @@FinalEdgeSet_4 += alias_schema_4
        POST-ACCUM @@FinalVertexSet_1 += alias_schema_1
        POST-ACCUM @@FinalVertexSet_2 += alias_schema_2
        ;

  PRINT @@FinalEdgeSet_3;

  PRINT @@FinalEdgeSet_4;

  VertexSet_1 = { @@FinalVertexSet_1 };
  PRINT VertexSet_1["" as no_attributes];

  VertexSet_2 = { @@FinalVertexSet_2 };
  PRINT VertexSet_2["" as no_attributes];

}

''')


"Warning in query pattern_clinical_trials_disease (WARN-8): line 152, col 8\nIn PRINT clause, it contains an expression that doesn't have\nattribute/vertex-attached accumulator of vertex set. It will be printed\nrepeatedly the number of times as the given vertex set size.\nWarning in query pattern_clinical_trials_disease (WARN-8): line 155, col 8\nIn PRINT clause, it contains an expression that doesn't have\nattribute/vertex-attached accumulator of vertex set. It will be printed\nrepeatedly the number of times as the given vertex set size.\nSuccessfully created queries: [pattern_clinical_trials_disease]."

In [12]:

conn.gsql('INSTALL QUERY pattern_clinical_trials_disease')

'Start installing queries, about 1 minute ...\npattern_clinical_trials_disease query: curl -X GET \'http://127.0.0.1:9000/query/graph_drug_disease/pattern_clinical_trials_disease?disease=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [25]:
#print(conn.gsql('RUN QUERY pattern_clinical_trials("TRAZODONE")'))

Semantic Check Fails: the query pattern_clinical_trials doesn't exist!


In [ ]:
TODO: read drug and disease and gene files afin de separer les embeddings

In [28]:
#print(conn.gsql('RUN QUERY tg_embedding_cosine_similarity (("1003016809","PRESCRIBER"),"PRESCRIBER",1000,3,_)'))

In [26]:
drug = "\"TRAZODONE\""

get_disease_for_drug = conn.gsql('RUN QUERY pattern_clinical_trials_drugs('+drug+')')
VertexSet = json_results_processing(get_disease_for_drug, "drug")
print("drug used for the following disease",VertexSet)

list_pred_drug_disease = []
for disease in VertexSet:
    disease = "\""+ disease +"\""
    
    
    query = 'RUN QUERY tg_common_neighbors (('+drug+', "DRUG") , ('+disease+', "DISEASES"),["is_prescripted_for", "gene_disease_association", "drug_gene_association", "is_tested_for"], _)'
    
    get_common_neighbors = conn.gsql(query)
        
    
    #print(get_common_neighbors)
    json_object = json.loads(get_common_neighbors)
    #json_object = json.load(getPRESCRIBER)
    results = json_object['results']
    #print(drug, disease, results[0]['closeness'])
    list_pred_drug_disease.append((drug, disease, results[0]['closeness']))
    list_pred_drug_disease = sorted(list_pred_drug_disease, key=lambda tup: tup[2], reverse=True)
list_pred_drug_disease

drug used for the following disease set()


[]

In [14]:
def json_results_processing(get_patterns, type_node):
    if(type_node == "drug"):
        Vertex_type = 'VertexSet_2'
        
    if(type_node == "disease"):
        Vertex_type = 'VertexSet_1'
        
    
    VertexList = []
    json_object = json.loads(get_patterns)
    #json_object
    results = json_object['results']
    for elem in results:
        for key, value in elem.items():
            #print(key)
            if(key == Vertex_type):
                #print(value)
                for i in value:
                    VertexList.append(i['v_id'])


            #if(key == '@@FinalEdgeSet_3' or key == '@@FinalEdgeSet_4'):
            #    for i in value:
            #        print(i['to_id'])
    VertexSet = set(VertexList)
    return VertexSet

def getNeighborsVertexSet(drug_or_disease, type_node):
    drug_or_disease = "\""  + drug_or_disease + "\"" 
    if(type_node == "drug"):
        query = 'RUN QUERY pattern_clinical_trials_drugs('+drug_or_disease+')'
    if(type_node == "disease"):
        query = 'RUN QUERY pattern_clinical_trials_disease('+drug_or_disease+')'
    get_neighbors = conn.gsql(query)
    #print(get_neighbors)
    VertexSet = json_results_processing(get_neighbors, type_node)
    return VertexSet

def getNodes(type_node):
    if(type_node == "drug"):
        query = 'RUN QUERY getDISEASES()'
        filename = "/home/tigergraph/getDISEASES.txt"
    if(type_node == "disease"):
        query = 'RUN QUERY getDRUGS()'
        filename = "/home/tigergraph/getDRUGS.txt"
        
    #if(os.path.exists(filename)==False):
    conn.gsql(query)
        
    getNodes = []
    with open(filename) as file:
        for line in file: 
            line = line.strip() 
            getNodes.append(line) 

        getNodes = set(getNodes)
    return getNodes

def diff_Vertex(set1, set2):
    diff_set = set1 - set2
    if '' in diff_set:
        diff_set.remove('')
    
    return diff_set

def run_common_neighbors_algo(query):
    
    get_common_neighbors = conn.gsql(query)
    
    json_object = json.loads(get_common_neighbors)
    results = json_object['results']
    return results
    
def predictions(drug_or_disease, type_node, diff_set, top_k):
    if "\"" in drug_or_disease:
        drug_or_disease = drug_or_disease.replace("\"", "'")
        print(drug_or_disease)
        drug_or_disease = drug_or_disease.replace("'", "\\'")
    drug_or_disease = "\""+ drug_or_disease +"\""
    list_pred_drug_disease = []
    
    if (type_node == "drug"): # predict disease for a drug
        
        for disease in diff_set:
            #if "\"" in disease:
            #    disease = disease.replace("\"", "'")
                
            #    disease = disease.replace("'", "\\'")
            #    print(disease)
                
            #    disease = disease.replace("\"", "\\\"")
            disease = "\""+ disease +"\""

            #print(drug, disease)
            query = 'RUN QUERY tg_common_neighbors (('+drug_or_disease+', "DRUG") , ('+disease+', "DISEASES"),["is_prescripted_for", "gene_disease_association", "drug_gene_association", "is_tested_for"], _)'
            
            results = run_common_neighbors_algo(query)
            list_pred_drug_disease.append((drug_or_disease, disease, results[0]['closeness']))
        list_pred_drug_disease = sorted(list_pred_drug_disease, key=lambda tup: tup[2], reverse=True)
            
    if (type_node == "disease"): # predict drug for a disease
        
        for drug in diff_set:
            #if "\"" in drug:
            #    drug = drug.replace("\"", "'")
                
            #    drug = drug.replace("'", "\\'")
            #    print(drug)
                
            drug = "\""+ drug +"\""

            #print(drug)
            #if("HYDROXYETHYLSTARCH" not in drug):
            query = 'RUN QUERY tg_common_neighbors (('+drug+', "DRUG") , ('+drug_or_disease+', "DISEASES"),["is_prescripted_for", "gene_disease_association", "drug_gene_association", "is_tested_for"], _)'
            #print(query)
            results = run_common_neighbors_algo(query)
            list_pred_drug_disease.append((drug, drug_or_disease, results[0]['closeness']))
        list_pred_drug_disease = sorted(list_pred_drug_disease, key=lambda tup: tup[2], reverse=True)


    
    return list_pred_drug_disease[:top_k]



In [15]:
#disease_name = ";NERVOUS_SYSTEMBONE_CYSTSANOPHTHALMOSPROSTATITISCOFFIN-LOWRY_SYNDROME"
disease_name = "VASCULAR_DISEASES"
neighbors_drugs = getNeighborsVertexSet(disease_name, "disease")
print(neighbors_drugs)
set_drugs = getNodes("disease")
print(len(set_drugs))
diff_drugs = diff_Vertex(set_drugs, neighbors_drugs)
print(len(diff_drugs))
predictions(disease_name, "disease", diff_drugs, 10)

set()
59671
59670


[('"BISPHENOL_A"', '"VASCULAR_DISEASES"', 23962),
 ('"TETRACHLORODIBENZODIOXIN"', '"VASCULAR_DISEASES"', 17111),
 ('"VALPROIC_ACID"', '"VASCULAR_DISEASES"', 16488),
 ('"ACETAMINOPHEN"', '"VASCULAR_DISEASES"', 14385),
 ('"TOBACCO_SMOKE_POLLUTION"', '"VASCULAR_DISEASES"', 12162),
 ('"AFLATOXIN_B1"', '"VASCULAR_DISEASES"', 12088),
 ('"SODIUM_ARSENITE"', '"VASCULAR_DISEASES"', 11851),
 ('"FOLIC_ACID"', '"VASCULAR_DISEASES"', 9865),
 ('"ETHINYL_ESTRADIOL"', '"VASCULAR_DISEASES"', 9786),
 ('"CYCLOSPORINE"', '"VASCULAR_DISEASES"', 9305)]

In [ ]:
conn.gsql('''
CREATE QUERY pattern_get_relations_of_disease(String disease) FOR GRAPH graph_drug_disease API("v2") SYNTAX v2 {

  SetAccum<string> @@VertexSet_2_idList;
  SetAccum<vertex> @@VertexSet_2_vertexList;
  SetAccum<edge> @@FinalEdgeSet_4;
  SetAccum<edge> @@FinalEdgeSet_5;
  SetAccum<edge> @@FinalEdgeSet_7;
  SetAccum<edge> @@FinalEdgeSet_6;
  SetAccum<vertex<DRUG>> @@FinalVertexSet_1;
  SetAccum<vertex<DISEASES>> @@FinalVertexSet_2;
  SetAccum<vertex<GENE>> @@FinalVertexSet_3;

  @@VertexSet_2_idList += disease;
  @@VertexSet_2_vertexList = to_vertex_set(@@VertexSet_2_idList, "DISEASES");
  VertexSet_2 (DISEASES) = { @@VertexSet_2_vertexList };
  VertexSet_1 = 
        SELECT alias_schema_1
        FROM DRUG:alias_schema_1 -(drug_is_tested_for:alias_schema_4)- VertexSet_2:alias_schema_2,
             DRUG:alias_schema_1 -(is_prescripted_for:alias_schema_5)- VertexSet_2:alias_schema_2,
             DRUG:alias_schema_1 -(drug_gene_association:alias_schema_7)- GENE:alias_schema_3,
             GENE:alias_schema_3 -(gene_disease_association:alias_schema_6)- VertexSet_2:alias_schema_2
        ACCUM @@FinalEdgeSet_4 += alias_schema_4,
              @@FinalEdgeSet_5 += alias_schema_5,
              @@FinalEdgeSet_7 += alias_schema_7,
              @@FinalEdgeSet_6 += alias_schema_6
        POST-ACCUM @@FinalVertexSet_1 += alias_schema_1
        POST-ACCUM @@FinalVertexSet_2 += alias_schema_2
        POST-ACCUM @@FinalVertexSet_3 += alias_schema_3
        ;

  PRINT @@FinalEdgeSet_4;

  PRINT @@FinalEdgeSet_5;

  PRINT @@FinalEdgeSet_7;

  PRINT @@FinalEdgeSet_6;

  VertexSet_1 = { @@FinalVertexSet_1 };
  PRINT VertexSet_1["" as no_attributes];

  VertexSet_2 = { @@FinalVertexSet_2 };
  PRINT VertexSet_2["" as no_attributes];

  VertexSet_3 = { @@FinalVertexSet_3 };
  PRINT VertexSet_3[
    VertexSet_3.GeneSymbol AS GeneSymbol,
    VertexSet_3.GeneName AS GeneName,
    VertexSet_3.AltGeneIDs AS AltGeneIDs,
    VertexSet_3.Synonyms AS Synonyms,
    VertexSet_3.BioGRIDIDs AS BioGRIDIDs,
    VertexSet_3.PharmGKBIDs AS PharmGKBIDs,
    VertexSet_3.UniProtIDs AS UniProtIDs
  ];

}

''')
conn.gsql('INSTALL QUERY pattern_get_relations_of_disease')

In [ ]:
#Parallel version

In [6]:
query = 'RUN QUERY getDISEASES()'
getDISEASES = []
filename = "/home/tigergraph/getDISEASES.txt"
#if(os.path.exists(filename)==False):
conn.gsql(query)
with open(filename) as file:
    for line in file: 
        line = line.strip() 
        getDISEASES.append(line) 

    getDISEASES = set(getDISEASES)
getDISEASES = list(getDISEASES)
getDISEASES.remove('')
getDISEASES[:10]

['ATRIOVENTRICULAR_BLOCK',
 'RETINITIS_PIGMENTOSA_69',
 'ALTERNATING_HEMIPLEGIA_OF_CHILDHOOD',
 'SENIOR-LOKEN_SYNDROME_5',
 'HEMOSPERMIA',
 'KURU',
 'HYPERFERRITINEMIA',
 'WAARDENBURG_SYNDROME_TYPE_2',
 'SPONDYLITIS',
 'PROTOZOAN_INFECTIONS']

In [21]:
import time
import math
from multiprocessing import Pool
from multiprocessing import freeze_support
from pathlib import Path

folder = "../results/"
Path(folder).mkdir(parents=True, exist_ok=True)

'''Define function to run mutiple processors and pool the results together'''
def run_multiprocessing(func, i, n_processors):
    with Pool(processes=n_processors) as pool:
        return pool.map(func, i)

def sequential_prediction_algo(disease_name):
    print(disease_name)
    neighbors_drugs = getNeighborsVertexSet(disease_name, "disease")
    #print(neighbors_drugs)
    set_drugs = getNodes("disease")
    #print(len(set_drugs))
    #neighbors_drugs.remove('')
    #set_drugs.remove('')
    diff_drugs = diff_Vertex(set_drugs, neighbors_drugs)
    #print(len(diff_drugs))
    results = predictions(disease_name, "disease", diff_drugs, 10)
    folder_disease_name = folder+disease_name+"/"
    Path(folder_disease_name).mkdir(parents=True, exist_ok=True)
    f = open(folder_disease_name+"predictions"+".txt", "w")
    for res in results:
        f.write(res+"\n")
    f.close()
        
    
start = time.clock()

'''
set up parameters required by the task
'''

n_processors = 30
list_disease = getDISEASES

'''
pass the task function, followed by the parameters to processors
'''
out = run_multiprocessing(sequential_prediction_algo, list_disease, n_processors)

print("Input length: {}".format(len(list_disease)))
print("Output length: {}".format(len(out)))
print("Mutiprocessing time: {}mins\n".format((time.clock()-start)/60))
print("Mutiprocessing time: {}secs\n".format((time.clock()-start)))

ATRIOVENTRICULAR_BLOCKRETT_SYNDROMENEUROGENIC_BOWELINTRACRANIAL_HYPERTENSIONGASTROSCHISISRETINITIS_PIGMENTOSA_40LACRIMAL_DUCT_OBSTRUCTION_BRAIN_INVASIVEVITAMIN_D_HYDROXYLATION-DEFICIENT_RICKETS
SPASTIC_PARAPLEGIA-50
HYPODERMYIASIS

_INHALATIONNEUROASPERGILLOSISHYPERCAROTENEMIA_AND_VITAMIN_A_DEFICIENCYADENOMA
AGAMMAGLOBULINEMIA_7

HOOKWORM_INFECTIONS

VASCULAR_SYSTEM_INJURIESACRO-OSTEOLYSISSPASTIC_ATAXIA_5INFRATENTORIAL_NEOPLASMS
INTRACRANIAL_HEMORRHAGESPINA_BIFIDA_CYSTICA
ACUTE_PAIN

HEART_NEOPLASMS
PYRUVATE_DEHYDROGENASE_COMPLEX_DEFICIENCY_DISEASEHEMATOLOGIC_NEOPLASMS

MUCOLIPIDOSIS_II_ALPHA/BETA






SODIUM_SERUM_LEVEL_QUANTITATIVE_TRAIT_LOCUS_1






CHONDROSARCOMA
VENOUS_INSUFFICIENCY
RED-CELL_APLASIAATAXIA_TELANGIECTASIA_LIKE_DISORDERGLAUCOMA_1
HYPERMETHIONINEMIACLEFT_PALATEPANCREATIC_CYST
PERIODONTAL_DISEASES
EPISODIC_ATAXIA
VON_HIPPEL-LINDAU_DISEASE


RETINITIS_PIGMENTOSA_58


SERRATIA_INFECTIONSSPASTIC_PARAPLEGIAPROGERIABARBER_SAY_SYNDROMEPYRUVATE_CARBOXYLASE_DEFICIENCY_DISEAS

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
s = set()
s.add('3')
s.add('2')
#s.add('')

if '' in s:
    s.remove('')
s

{'2', '3'}

In [427]:
list_pred_drug_disease[:10]

[('"BISPHENOL_A"', '"VASCULAR_DISEASES"', 23962),
 ('"BENZO(A)PYRENE"', '"VASCULAR_DISEASES"', 18258),
 ('"TETRACHLORODIBENZODIOXIN"', '"VASCULAR_DISEASES"', 17111),
 ('"VALPROIC_ACID"', '"VASCULAR_DISEASES"', 16488),
 ('"ACETAMINOPHEN"', '"VASCULAR_DISEASES"', 14385),
 ('"TOBACCO_SMOKE_POLLUTION"', '"VASCULAR_DISEASES"', 12162),
 ('"AFLATOXIN_B1"', '"VASCULAR_DISEASES"', 12088),
 ('"SODIUM_ARSENITE"', '"VASCULAR_DISEASES"', 11851),
 ('"FOLIC_ACID"', '"VASCULAR_DISEASES"', 9865),
 ('"ETHINYL_ESTRADIOL"', '"VASCULAR_DISEASES"', 9786)]

In [493]:
drug = "\"PROPHYLACTIC\"_DELAYED_ACTIVATED_DONOR_LYMPHOCYTE_INFUSION"
print(drug)
#drug = "\""
#print(drug)
#drug = drug.replace("\\", "\\\\")
drug = drug.replace("\"", "'")
print(drug)
drug = drug.replace("'", "\\'")
#drug = drug.replace("\\\"", "\"")
print(drug)

"PROPHYLACTIC"_DELAYED_ACTIVATED_DONOR_LYMPHOCYTE_INFUSION
'PROPHYLACTIC'_DELAYED_ACTIVATED_DONOR_LYMPHOCYTE_INFUSION
\'PROPHYLACTIC\'_DELAYED_ACTIVATED_DONOR_LYMPHOCYTE_INFUSION


In [416]:
print("\\")

\


In [447]:
#print(conn.gsql('RUN QUERY tg_jaccard_nbor_ss ("PRESCRIBER", "prescript_drug", "prescript_drug",  5, _, _, "/home/tigergraph/test/test.csv")'))
#print(conn.gsql('RUN QUERY tg_jaccard_nbor_ap_batch (5, "PRESCRIBER", "DRUG","prescript_drug", "prescript_drug",_, 15, 15, _, -1, "/home/tigergraph/test/test.csv")'))
#print(conn.gsql('RUN QUERY tg_jaccard_nbor_ap_batch (5, ["PRESCRIBER", "DRUG"],["PRESCRIBER"],["prescript_drug"], _,_, 15, 15, _, 1000, "/home/tigergraph/test/test.csv")'))
#conn.gsql('RUN QUERY tg_jaccard_nbor_ap_batch (5, ["PRESCRIBER", "DRUG"],["PRESCRIBER"],["prescript_drug"], _,_, 15, 15, _, 5, "/home/tigergraph/test/result_test.csv")')

#print(conn.gsql('RUN QUERY tg_jaccard_nbor_ap_batch (5, "DISEASES", "DRUG","is_prescripted_for", "is_prescripted_for",_, _, _, _, -1, _)'))

#"/home/tigergraph/test/test.csv"

 


In [ ]:
#list_nodes = []
#json_object = json.loads(getPRESCRIBER)
#json_object = json.load(getPRESCRIBER)
#results = json_object['results']
#for elem in results[0]['VertexSet_1']:
    #print(elem)
    #print(elem['v_id'])
#    list_nodes.append(elem['v_id'])

import timeit
import numpy as np

start = timeit.default_timer()

#Your statements here


node = "1003016809"

list_common_neighbors = []    
#tmp = list_nodes[:5]
tmp = np.array(getPRESCRIBER)
#tmp = getPRESCRIBER
i = 0

f = open("common_neighbors.csv", "w")
sep = ","


while i < len(tmp):
    #print(tmp[i])
    if(i%100000 == 0):
        print(i)
    
    get_common_neighbors = conn.gsql('RUN QUERY tg_common_neighbors (('+tmp[i]+', "PRESCRIBER") , ('+node+', "PRESCRIBER"), "prescript_drug", _)')
    json_object = json.loads(get_common_neighbors)
    results = json_object['results']
    score = results[0]['closeness']
    #list_common_neighbors.append((tmp[i], node, score))
    
    f.write(str(tmp[i])+sep+str(node)+sep+str(score))
    f.write("\n")
    #print(i, j, tmp[i], tmp[j], score)

    i += 1
f.close()
stop = timeit.default_timer()

print('Time: ', stop - start)  

list_common_neighbors[:2]   
